In [14]:
import json

with open('./data/paragraphs_v1.json', 'r') as f:
    paragraphs = json.load(f)
with open('/home/xilini/par-gen/01-par-gen/data/splits.json', 'r') as i:
    splits = json.load(i)

In [15]:
paragraphs

[{'url': 'https://cs.stanford.edu/people/rak248/VG_100K/2356347.jpg',
  'image_id': 2356347,
  'paragraph': 'A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. '},
 {'url': 'https://cs.stanford.edu/people/rak248/VG_100K/2317429.jpg',
  'image_id': 2317429,
  'paragraph': 'A white round plate is on a table with a plastic tablecloth on it.  Two foil covered food halves are on the white plate along with a serving of golden yellow french fries.  Next to the white plate in a short,  topless, plastic container is a white sauce.  Diagonal to the white plate are the edges of several other stacked plates.  There are black shadows reflected on the table.'},
 {'url': 'https://cs.stanford.edu/people/rak248/VG_100K_2/2414610.jpg',
  'image_id': 2414610,
  'paragraph': 'A woman in a blue tennis outfit stands on a green tennis court. She is swinging a blue tennis racket. There is 

In [16]:
splits

{'2406949': 'train',
 '2387215': 'train',
 '2372347': 'train',
 '2369202': 'train',
 '2368287': 'train',
 '2352527': 'train',
 '2358049': 'train',
 '2380206': 'train',
 '2375670': 'train',
 '2389065': 'train',
 '2358054': 'train',
 '2389824': 'train',
 '2403652': 'train',
 '2383990': 'train',
 '2389326': 'train',
 '2397541': 'train',
 '2391306': 'train',
 '2403265': 'train',
 '2322521': 'train',
 '2349382': 'train',
 '2409025': 'train',
 '2361341': 'train',
 '2411159': 'train',
 '2372658': 'train',
 '2350124': 'train',
 '2331849': 'train',
 '2405881': 'train',
 '2362382': 'train',
 '2381565': 'train',
 '2408056': 'train',
 '2382766': 'train',
 '2397629': 'train',
 '2406837': 'train',
 '2411655': 'train',
 '2316759': 'train',
 '2368662': 'train',
 '2317630': 'train',
 '2397566': 'train',
 '2387421': 'train',
 '2406786': 'train',
 '2359841': 'train',
 '2402212': 'train',
 '2393855': 'train',
 '4505': 'train',
 '2364789': 'train',
 '2343208': 'train',
 '2385643': 'train',
 '387': 'train',

In [27]:
curr_sentence_id = 0
from spacy.lang.en import English # updated
from autocorrect import Speller
spell = Speller(lang='en')

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated

all_images = {}
all_images['images'] = []

for par_num, paragraph in enumerate(paragraphs, 1):
    filepath = paragraph['url'].split('/')[-2]
    filename = paragraph['url'].split('/')[-1]
    imgid = par_num
    split = splits[str(filename.split('.')[0])]
    
    paragraph = paragraph['paragraph']
    doc = nlp(paragraph)
    sentences = [sent for sent in doc.sents]
    
    sent_dict = {}
    sent_dict['filepath'] = filepath
    sent_dict['filename'] = filename
    sent_dict['imgid'] = imgid
    sent_dict['split'] = split
    sent_dict['sentences'] = []
    
    sent_ids = []
    tokens_dict = {}
    
    for s in sentences:
        tokens_dict = {}
        raw_sentence = s
        #print(raw_sentence)
        tokens = [token.orth_.lower().rstrip() for token in raw_sentence]
        tokens = [t for t in tokens if t != '']
        # check mispelled
        tokens = [spell(t) for t in tokens]
        if len(tokens) != 0:
            tokens_dict['tokens'] = tokens
            tokens_dict['raw'] = str(raw_sentence)
            tokens_dict['imgid'] = imgid
            tokens_dict['sentid'] = curr_sentence_id
            sent_ids.append(curr_sentence_id)
            curr_sentence_id += 1
        
            sent_dict['sentences'].append(tokens_dict)

    sent_dict['stanford_par_id'] = int(filename.split('.jpg')[0])
    sent_dict['sentids'] = sent_ids
    all_images['images'].append(sent_dict)
    #break
    #print(split)

In [28]:
all_images['images'][-1]

{'filepath': 'VG_100K',
 'filename': '2344677.jpg',
 'imgid': 19561,
 'split': 'train',
 'sentences': [{'tokens': ['the',
    'women',
    'is',
    'wearing',
    'a',
    'sweater',
    '.'],
   'raw': 'The women is wearing a sweater.',
   'imgid': 19561,
   'sentid': 111789},
  {'tokens': ['the', 'sweater', 'is', 'red', '.'],
   'raw': 'The sweater is red.',
   'imgid': 19561,
   'sentid': 111790},
  {'tokens': ['the',
    'women',
    'is',
    'holding',
    'a',
    'dog',
    'that',
    'is',
    'brown',
    '.'],
   'raw': 'The women is holding a dog that is brown.',
   'imgid': 19561,
   'sentid': 111791},
  {'tokens': ['the', 'dog', 'is', 'small', '.'],
   'raw': 'The dog is small.',
   'imgid': 19561,
   'sentid': 111792},
  {'tokens': ['the', 'dog', 'is', 'touching', 'the', 'railing', '.'],
   'raw': 'The dog is touching the railing.',
   'imgid': 19561,
   'sentid': 111793},
  {'tokens': ['the', 'railing', 'is', 'white', '.'],
   'raw': 'The railing is white.',
   'imgid

In [29]:
with open('/home/xilini/par-gen/01-par-gen/data/dataset_paragraphs_v1.json', 'w') as w:
    json.dump(all_images, w)

In [30]:
with open('/home/xilini/par-gen/01-par-gen/data/dataset_paragraphs_v1.json', 'r') as w:
    imgs = json.load(w)

In [31]:
len(imgs['images'])

19561

In [33]:
for i in imgs['images']:
    for s in i['sentences']:
        #print(s)
        if len(s['tokens']) == 0:
            print(s['tokens'], len(s['tokens']))
            print(s)